In [1]:
from astropy.io import fits
from astropy.table import vstack
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
import seaborn as sns
import scipy.stats
import seaborn as sns
from numpy.random import normal
import seaborn as sns
import numpy
import numexpr

In [72]:
#import s5 data
s5 = fits.open('s5cat_dr3_2.fits')
data = np.array(s5[1].data)
df = pd.DataFrame(data).reset_index(drop=True)
print(df['object_name'])

0              b'Styx'
1              b'Styx'
2              b'Styx'
3              b'Styx'
4              b'Styx'
             ...      
84102        b'Orphan'
84103    b'Phlegethon'
84104    b'Phlegethon'
84105    b'Phlegethon'
84106    b'Phlegethon'
Name: object_name, Length: 84107, dtype: object


In [62]:
#import gaia cluster data 
clusters = np.genfromtxt('GaiaTools-master/result.txt', delimiter='\t', skip_header=1, dtype=float)
print(clusters[0])


[        nan  6.0240e+00 -7.2081e+01  4.5200e+00  3.0000e-02 -1.7450e+01
  1.6000e-01  1.2200e+01  4.0000e+01  5.2590e+00 -2.5430e+00  2.3200e-01
  2.5000e-02  2.5000e-02 -0.0000e+00  9.0000e-03]


In [48]:
#coordinate transformation algorithms
from numpy import sin,cos,deg2rad,rad2deg,arctan2,sqrt

def cv_coord(a,b,c,fr=None,to=None,degr=False):
	if degr:
		degrad = deg2rad
		raddeg = rad2deg
	else:
		degrad = lambda x: x
		raddeg = lambda x: x
	if fr=='sph':
		x=c*cos(degrad(a))*cos(degrad(b))
		y=c*sin(degrad(a))*cos(degrad(b))
		z=c*sin(degrad(b))
	elif fr=='rect':
		x=a
		y=b
		z=c
	elif fr is None:
		raise Exception('You must specify the input coordinate system')
	else:
		raise Exception('Unknown input coordinate system')
	if to=='rect':
		return (x,y,z)
	elif to=='sph':
		ra = raddeg(arctan2(y,x))
		dec = raddeg(arctan2(z,sqrt(x**2+y**2)))
		rad = sqrt(x**2+y**2+z**2)
		return (ra,dec,rad)
	elif to is None:
		raise Exception('You must specify the output coordinate system')
	else:
		raise Exception('Unknown output coordinate system')


def torect(ra,dec):
	x=numexpr.evaluate('cos(ra/57.295779513082323)*cos(dec/57.295779513082323)')
	y=numexpr.evaluate('sin(ra/57.295779513082323)*cos(dec/57.295779513082323)')
	z=numexpr.evaluate('sin(dec/57.295779513082323)')
	return x,y,z

def fromrect(x,y,z):
	ra=numexpr.evaluate('arctan2(y,x)*57.295779513082323')
	dec=numexpr.evaluate('57.295779513082323*arctan2(z,sqrt(x**2+y**2))')
	return ra,dec

def sphere_rotate(ra, dec, rapol, decpol, ra0, revert=False):
	""" rotate ra,dec to a new spherical coordinate system where the pole is 
		at rapol,decpol and the zeropoint is at ra=ra0 
		revert flag allows to reverse the transformation
	"""

	x,y,z=torect(ra,dec)

	tmppol=cv_coord(rapol,decpol,1,degr=True,fr='sph',to='rect') #pole axis
	tmpvec1=cv_coord(ra0,0,1,degr=True,fr='sph',to='rect') #x axis
	tmpvec1=numpy.array(tmpvec1)

	tmpvec1[2]=(-tmppol[0]*tmpvec1[0]-tmppol[1]*tmpvec1[1])/tmppol[2]
	tmpvec1/=numpy.sqrt((tmpvec1**2).sum())
	tmpvec2=numpy.cross(tmppol,tmpvec1) # y axis 

	if not revert:
		Axx,Axy,Axz=tmpvec1
		Ayx,Ayy,Ayz=tmpvec2
		Azx,Azy,Azz=tmppol
	else:
		Axx,Ayx,Azx=tmpvec1
		Axy,Ayy,Azy=tmpvec2
		Axz,Ayz,Azz=tmppol
	xnew = numexpr.evaluate('x*Axx+y*Axy+z*Axz')
	ynew = numexpr.evaluate('x*Ayx+y*Ayy+z*Ayz')
	znew = numexpr.evaluate('x*Azx+y*Azy+z*Azz')
	
	del x,y,z
	new_ra, new_dec = fromrect(xnew,ynew,znew)
	return new_ra,new_dec



In [49]:
#cluster list
cluster_candidate= [
 'Antlia2',
 'Crater2',
 'NGC1261',
 'NGC1851',
 'NGC1904',
 'NGC3201',
 'NGC4590',
 'NGC6316',
 'NGC7078',
 'NGC7089',
 'Pal13',
 'Pal15',
 'Pal5',
]

test for using astropy.match_coordinates

In [74]:
cluster_name = 'NGC1904'


field = df[df['object_name'].str.contains(cluster_name)]

"""
streams = field[field['priority']>8].reset_index(drop=True)

count_plots = 0
if len(streams['ra'])>0:
        
    original_ra = np.array(streams['ra'])
    original_dec = np.array(streams['dec'])

c = SkyCoord(ra=original_ra*u.degree, dec = original_dec*u.degree)
catalog = SkyCoord(ra=clusters[:,1] , dec=clusters[:,2] )
idx = c.match_to_catalog_sky(c.frame,catalog.frame)
"""


TypeError: Cannot use .str.contains with values of inferred dtype 'bytes'.